In [162]:
import pandas as pd
from tqdm import tqdm
import requests
import time
S = requests.Session()

In [163]:
data = pd.read_csv("wiki_text.csv")

del data['Unnamed: 0']

data['location'] = ''
data['inception'] = ''
data['image'] = ''
data['discovery'] = ''
data['country'] = ''

data.head()

,category,entity,text,title,location,inception,image,discovery,country
0,Category:Archaeological artefact types,Q2517447,An adze (; alternative spelling: adz) is a cut...,Adze,,,,,
1,Category:Archaeological artefact types,Q262026,"An andiron or firedog, fire-dog or fire dog is...",Andiron,,,,,
2,Category:Archaeological artefact types,Q1196338,"The ard, ard plough, or scratch plough is a si...",Ard (plough),,,,,
3,Category:Archaeological artefact types,Q45922,An arrow is a fin-stabilized projectile that i...,Arrow,,,,,
4,Category:Archaeological artefact types,Q220659,"An artifact, or artefact (see American and Bri...",Artifact (archaeology),,,,,


In [180]:
def scrape_page_wikidata(entity):
    PARAMS = {
        "format": "json",
        "query":
        '\
            SELECT ?wdLabel ?ps_Label ?wdpqLabel ?pq_Label {\
              VALUES (?company) {(wd:' + entity + ')}\
                    ?company ?p ?statement .\
                    ?statement ?ps ?ps_ .\
                    ?wd wikibase:claim ?p.\
                    ?wd wikibase:statementProperty ?ps.\
                    OPTIONAL {\
                      ?statement ?pq ?pq_ .\
                      ?wdpq wikibase:qualifier ?pq .\
                    }\
                  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }\
                }\
            ORDER BY ?wd ?statement ?ps_'
    }
    R = S.get(url="https://query.wikidata.org/sparql", params=PARAMS)
    try:
        wikidata_json = R.json()
        results = {}
        for obj in wikidata_json['results']['bindings']:
            key = obj['wdLabel']['value']
            val = obj['ps_Label']['value']
            results[key] = val
        return results
    except Exception as e:
        print("An exception occurred at page ", entity)
        print(R.headers)
        print(R.text)
        time.sleep(10)
        # Workaround to not be banned
        return scrape_page_wikidata(entity)

In [177]:
scrape_page_wikidata("Q2517447")    

{'image': 'http://commons.wikimedia.org/wiki/Special:FilePath/TroughAdze.jpg',
 'AAT ID': '300023553',
 'described by source': 'Ottův slovník naučný',
 'Encyclopædia Britannica Online ID': 'technology/adz',
 'subclass of': 'woodworking tool',
 'Commons category': 'Adzes',
 'JSTOR topic ID': 'adzes',
 'PACTOLS thesaurus ID': 'pcrtTg7ubUZSuL',
 'BNCF Thesaurus ID': '48160',
 'Freebase ID': '/m/0swv4'}

In [166]:
data.iloc[1]['entity']

'Q262026'

In [ ]:
for idx in tqdm(data.index):
    if idx < 6207:
        continue
    wikidata = scrape_page_wikidata(data.iloc[idx]['entity'])
    data.at[idx, 'location'] = wikidata.get('location', '')
    data.at[idx, 'inception'] = wikidata.get('inception', '')
    data.at[idx, 'image'] = wikidata.get('image', '')
    data.at[idx, 'discovery'] = wikidata.get('location of discovery', '')
    data.at[idx, 'country'] = wikidata.get('country', '')
data.to_csv("wikidata_and_text.csv")


 73%|███████▎  | 6208/8551 [00:00<00:00, 16152.57it/s]

An exception occurred at page  Q258695
{'Date': 'Sat, 13 Jul 2019 21:35:38 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '778101287', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h1

An exception occurred at page  Q1726492
{'Date': 'Sat, 13 Jul 2019 21:36:25 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '788028093', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h

An exception occurred at page  Q7196853
{'Date': 'Sat, 13 Jul 2019 21:37:16 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '794659374', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h

An exception occurred at page  Q740755
{'Date': 'Sat, 13 Jul 2019 21:38:14 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '789914346', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h1

...

An exception occurred at page  Q1496630
{'Date': 'Sat, 13 Jul 2019 22:19:05 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '810777456', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h


 99%|█████████▊| 8444/8551 [43:41<00:26,  4.09it/s]

An exception occurred at page  Q1418122
{'Date': 'Sat, 13 Jul 2019 22:19:17 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '827534064', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h


 99%|█████████▉| 8458/8551 [43:52<00:27,  3.36it/s]

An exception occurred at page  Q6737212
{'Date': 'Sat, 13 Jul 2019 22:19:29 GMT', 'Content-Type': 'text/html; charset=utf-8', 'Content-Length': '1942', 'Connection': 'keep-alive', 'Server': 'Varnish', 'X-Varnish': '814019311', 'X-Cache': 'cp3030 int', 'X-Cache-Status': 'int-front', 'Server-Timing': 'cache;desc="int-front"', 'Strict-Transport-Security': 'max-age=106384710; includeSubDomains; preload', 'X-Analytics': 'WMF-Last-Access=13-Jul-2019;WMF-Last-Access-Global=13-Jul-2019;https=1', 'X-Client-IP': '93.77.143.224'}
<!DOCTYPE html>
<html lang="en">
<meta charset="utf-8">
<title>Wikimedia Error</title>
<style>
* { margin: 0; padding: 0; }
body { background: #fff; font: 15px/1.6 sans-serif; color: #333; }
.content { margin: 7% auto 0; padding: 2em 1em 1em; max-width: 640px; }
.footer { clear: both; margin-top: 14%; border-top: 1px solid #e5e5e5; background: #f9f9f9; padding: 2em 0; font-size: 0.8em; text-align: center; }
img { float: left; margin: 0 2em 2em 0; }
a img { border: 0; }
h

In [184]:
data.to_csv("wikidata_and_text.csv")

In [170]:
data['inception'].unique()

array(['', '-25000-01-01T00:00:00Z', '-0449-01-01T00:00:00Z',
       '-0600-01-01T00:00:00Z', '-0559-01-01T00:00:00Z',
       '-0570-01-01T00:00:00Z', '-0659-01-01T00:00:00Z',
       '-0800-01-01T00:00:00Z', '-1675-01-01T00:00:00Z',
       '-1100-01-01T00:00:00Z', '-0400-01-01T00:00:00Z',
       '-29000-01-01T00:00:00Z', '-1920-01-01T00:00:00Z',
       '-0565-01-01T00:00:00Z', '-0550-01-01T00:00:00Z',
       '-0560-01-01T00:00:00Z', '-0489-01-01T00:00:00Z',
       '-0640-01-01T00:00:00Z', '-0479-01-01T00:00:00Z',
       '-0580-01-01T00:00:00Z', '-0540-01-01T00:00:00Z',
       '-0700-01-01T00:00:00Z', '-0599-01-01T00:00:00Z',
       '-0500-01-01T00:00:00Z', '-0669-01-01T00:00:00Z',
       '-0539-01-01T00:00:00Z', '-0430-01-01T00:00:00Z',
       '-0150-01-01T00:00:00Z', '-0519-01-01T00:00:00Z',
       '-1400-01-01T00:00:00Z', '-4000-01-01T00:00:00Z',
       '1855-01-01T00:00:00Z', '-1749-01-01T00:00:00Z',
       '-2100-01-01T00:00:00Z', '1974-01-01T00:00:00Z',
       '-2200-01-01T00:00:0